## Waste Management using CNN Model ##

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm as tqdm
import cv2
import warnings
warnings.filterwarnings("ignore")

In [4]:
train_data="dataset\DATASET\DATASET\TRAIN"
test_data="dataset\DATASET\DATASET\TEST"


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation , BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array, load_img
from tensorflow.keras.utils import plot_model
from glob import glob


In [ ]:
x1_data=[]
y1_data=[]
for category in glob(train_data+'/*'):
    for file in tqdm.tqdm(glob(category+'/*')):
        img_array=cv2.imread(file)
        img_array=cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)
        x1_data.append(img_array)
        y1_data.append(category.split('/')[-1])
data=pd.DataFrame({"image":x1_data,"label":y1_data})

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
x2_data=[]
y2_data=[]
for category in glob(train_data+'/*'):
    for file in tqdm.tqdm(glob(category+'/*')):
        img_array=cv2.imread(file)
        img_array=cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
        x2_data.append(img_array)
        y2_data.append(category.split('/')[-1])
data1=pd.DataFrame({"image":x2_data,"label":y2_data})

In [ ]:
data1.head()

In [ ]:
data1.shape

In [ ]:
x2_data[0].shape

In [ ]:
colors=['#a0d157','#c48bb8']
plt.pie(data.label.value_counts() ,labels=['Organic','Recycle'],autopct='%0.2f%%',colors=colors,shadow=True, startangle=140)

### HYPER-PARAMETERS ##


In [1]:
image_size=(256,256)
batch_size=32
epochs=10

### PIXEL FINDING ###

In [ ]:
from PIL import Image

# Load the image
image_path = "D:\\project\\intership_waste_classification\\dataset\\DATASET\\TRAIN\\R\\R_44.jpg"
img = Image.open(image_path)

# Get dimensions
width, height = img.size
channels = len(img.getbands())  # Get number of channels (e.g., 'RGB' -> 3, 'L' -> 1)

# Calculate dimensionality
dimensionality = width * height * channels
print(f"Image Dimensionality: {dimensionality} width: {width} height: {height} channels: {channels}")


### DATASET PREPARATION ###

In [15]:
train_datagen = ImageDataGenerator(
    rescale=1./255,                  # Normalize pixel values to [0, 1]
    rotation_range=20,               # Random rotation within ±20 degrees
    width_shift_range=0.1,           # Random horizontal shift by 10%
    height_shift_range=0.1,          # Random vertical shift by 10%
    zoom_range=0.1,                  # Random zoom in/out by 10%
    horizontal_flip=True,            # Random horizontal flip
    brightness_range=(0.8, 1.2),     # Random brightness adjustment
    fill_mode="nearest"              # Fill empty pixels after transformation
)

test_datagen= ImageDataGenerator(rescale=1./255)



In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

In [20]:
# Define the model
model = Sequential([
    
    Conv2D(64, (5, 5), padding='same', activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2), strides=(2, 2)),

    
    Conv2D(64, (5, 5), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2), strides=(2, 2)),

    
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2), strides=(2, 2)),

   
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2), strides=(2, 2)),

    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)